# Colony Health Status Checks 
- making new spreadsheet to double check colony health statuses 

In [1]:
library(dplyr)
library(tidyverse)
library(ggplot2)
library(tidyr)
library(purrr)
library(stringr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.2     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
# upload colony data and delete first empty col 
colony <- read_csv("/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv", show_col_types = FALSE)
colony <- colony[, -1] 

New names:
• `` -> `...1`


In [6]:
head(colony)

Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,Size_Class,⋯,082024_Percentage,Notes_082024,122024_Condition,122024_Percentage,Notes_122024,062025_Condition,062025_Percentage,Notes_062025,immune_y/n,checked_colonies
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
6/21/19,CBC30N,1,349,1,SSID,1.5,0.50,right,5,⋯,NA,NA,Not_visited,NA,Diseased,Diseased,NA,NA,n,x
6/21/19,CBC30N,1,334,2,PAST,1.4,0.10,left,4,⋯,NA,sample lower colony (original colony); inc old mort; too small for immune,NA,NA,NA,Healthy,NA,NA,y,x
6/21/19,CBC30N,1,346,3,SSID,6.3,0.80,right,5,⋯,NA,90% CLP; some discoloration,DC,10%,30% mortality,Diseased,NA,NA,y,x
10/15/19,CBC30N,1,407,4,PSTR,6.5,0.50,right,3,⋯,NA,NA,Not_visited,NA,Dead,NA,NA,NA,n,x
6/24/19,CBC30N,1,394,5,SSID,4.3,1.20,left,5,⋯,NA,NA,DC,10%,80% mortality,Diseased,NA,NA,n,x
10/15/19,CBC30N,1,413,6,PSTR,8.3,1.75,left,NA,⋯,NA,NA,Healthy,NA,95% mortality,NA,NA,NA,n,x


In [ ]:
# checking colony data 
#    pivot colonydata - each row is a colony’s health status at given date, 
# headers: Transect, Colony, Date, Health Status, Checked (initials)

In [17]:
# help from chatgpt to extract the date portion from condition & percentage columns 
colony_checks<-colony %>%
# filter columns 
  select(-c(OldTagNum, Meter, Meters_90, Direction, Size_Class,
            MaxDiameter, Height), 
         -matches("Notes")) %>%
  pivot_longer(
      # select cols to pivot
    cols = matches("\\d{6}_(Condition|Percentage)"),
      #selects part of the column name (date and condition/%) and makes it new cols
    names_to = c("Date", ".value"),
    names_pattern = "(\\d{6})_(Condition|Percentage)"
  ) %>%
# optional: change date to real date format 
#   mutate(Date = as.Date(paste0(substr(Date, 1, 2), "/01/", substr(Date, 3, 6)), format = "%m/%d/%Y"))
    rename(check_condition_initials = checked_colonies) %>%
# add new blank column 
    mutate(check_condition_notes= NA)

head(colony_checks)

Date_InitialTag,Transect,TransectNum,NewTagNum,Species,Date_DocumentedDisease,Date_DocumentedMortality,immune_y/n,check_condition_initials,Date,Condition,Percentage,check_condition_notes
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
6/21/19,CBC30N,1,1,SSID,5/21/22,Diseased,n,x,062019,Healthy,NA,NA
6/21/19,CBC30N,1,1,SSID,5/21/22,Diseased,n,x,052022,Diseased,NA,NA
6/21/19,CBC30N,1,1,SSID,5/21/22,Diseased,n,x,122022,Diseased,NA,NA
6/21/19,CBC30N,1,1,SSID,5/21/22,Diseased,n,x,092023,Diseased,NA,NA
6/21/19,CBC30N,1,1,SSID,5/21/22,Diseased,n,x,112023,Not_Visited,NA,NA
6/21/19,CBC30N,1,1,SSID,5/21/22,Diseased,n,x,122023,Not_Visited,NA,NA
